In [ ]:
import csv
import torch
import pandas as pd
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Pre-processing data

In [ ]:
train_set=[]
i=0
with open('/content/gdrive/My Drive/Hindi to English MT/train.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
      if (i==0):
        i+=1
        continue
      train_set.append([row[1],row[2].lower()])
train_set[0:10] 

[['एल सालवाडोर मे, जिन दोनो पक्षों ने सिविल-युद्ध से वापसी ली, उन्होंने वही काम किये जो कैदियों की कश्मकश के निदान हैं।',
  "in el salvador, both sides that withdrew from their civil war took moves that had been proven to mirror a prisoner's dilemma strategy."],
 ['मैं उनके साथ कोई लेना देना नहीं है.', 'i have nothing to do with them.'],
 ['-हटाओ रिक.', 'fuck them, rick.'],
 ['क्योंकि यह एक खुशियों भरी फ़िल्म है.', "because it's a happy film."],
 ['The thought reaching the eyes...', 'the thought reaching the eyes...'],
 ['मैंने तुमे School से हटवा दिया .', 'i got you suspended.'],
 ['यह Vika, एक फूल है.', "it's a flower, vika."],
 ['पर मेरे लिए उसका यहुदी विरोधी होना उसके कार्यों को और भी प्रशंसनीय बनाता है क्योंकि उसके पास भी पक्षपात करने के वही कारण थे जो बाकी फौजियों के पास थे पर उसकी सच जानने और उसे बनाए रखने की प्रेरणा सबसे ऊपर थी',
  'but personally, for me, the fact that picquart was anti-semitic actually makes his actions more admirable, because he had the same prejudices, the 

In [ ]:
len(train_set) #size of original train dataset

102322

In [ ]:
#removing the punctuation, unnecessary spaces and expanding words like "can't" to "can not"
processing_dict={"\'re": "are","\'m":" am", "let\'s":"let us","\'s":" is","\'ve":" have","\'ll":" will",
    "\'re":" are", "don\'t":"do not","didn't":"did not","can\'t":"can not","couldn\'t":"could not",
    "wouldn\'t":"would not", "doesn\'t":"does not", "isn\'t":"is not","won\'t":"will not",
    "weren\'t":"were not","hadn\'t":"had not","aren\'t":"are not", "hasn\'t":"has not",
    "wasn\'t":"was not", "shouldn\'t":"should not","ain\'t":"am not","-":" ", "(":" ",")":" ","{":" ",
    "}":" ","[":" ", "]":" ",":":" ", "\'":" ","\"":" ","\&":" and ", ",":" ","#":" ", "/":" ","\\":" ",
    "♪":" ","\=":" ","¶":" ","~":" ","  ":" "}
for i in range(0,len(train_set)):
  for j in range(0,2):
    for (src,trg) in processing_dict.items():
      if src in train_set[i][j]:
        train_set[i][j]=train_set[i][j].replace(src,trg) 
    train_set[i][0]=train_set[i][0].replace("..."," ")
    train_set[i][0]=train_set[i][0].replace(".","|")
    train_set[i][1]=train_set[i][1].replace("...."," ")
    train_set[i][1]=train_set[i][1].replace("..."," ")
    train_set[i][1]=train_set[i][1].replace(".."," ")
with open('/content/gdrive/My Drive/Hindi to English MT/pre_processed_train.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(['hindi','english'])
    write.writerows(train_set)


In [ ]:
train_set[0:20]

[['एल सालवाडोर मे जिन दोनो पक्षों ने सिविल युद्ध से वापसी ली उन्होंने वही काम किये जो कैदियों की कश्मकश के निदान हैं।',
  'in el salvador both sides that withdrew from their civil war took moves that had been proven to mirror a prisoner is dilemma strategy.'],
 ['मैं उनके साथ कोई लेना देना नहीं है|', 'i have nothing to do with them.'],
 [' हटाओ रिक|', 'fuck them rick.'],
 ['क्योंकि यह एक खुशियों भरी फ़िल्म है|', 'because it is a happy film.'],
 ['The thought reaching the eyes ', 'the thought reaching the eyes '],
 ['मैंने तुमे School से हटवा दिया |', 'i got you suspended.'],
 ['यह Vika एक फूल है|', 'it is a flower vika.'],
 ['पर मेरे लिए उसका यहुदी विरोधी होना उसके कार्यों को और भी प्रशंसनीय बनाता है क्योंकि उसके पास भी पक्षपात करने के वही कारण थे जो बाकी फौजियों के पास थे पर उसकी सच जानने और उसे बनाए रखने की प्रेरणा सबसे ऊपर थी',
  'but personally for me the fact that picquart was anti semitic actually makes his actions more admirable because he had the same prejudices the same reason

## Creating Train and validation set

In [ ]:
n=len(train_set)
train_ratio=0.80
train_size=int(n*train_ratio)
val_size=int(n-train_size)
train_ds, val_ds = train_set[:train_size],train_set[train_size:]
len(train_ds), len(val_ds)

(81857, 20465)

In [ ]:
with open('/content/gdrive/My Drive/Hindi to English MT/train_ds.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(train_ds)
with open('/content/gdrive/My Drive/Hindi to English MT/validation_ds.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(val_ds)

## Training the Model

In [ ]:
import spacy #importing spacy for english tokenizer
eng= spacy.load("en")

In [ ]:
#installing Indic NLP packages for hindi tokenizer
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
!pip install Morfessor
!pip install torchtext==0.6.0

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 24.98 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Checking out files: 100% (28/28), done.
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (654/654), done.
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 1.2MB 8.4MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [ ]:
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library" # path to local git repo for Indic NLP library
INDIC_NLP_RESOURCES="/content/indic_nlp_resources" # path to local git repo for Indic NLP Resources

In [ ]:
#installing the required packages
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, BucketIterator, Dataset, Example

In [ ]:
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()

In [ ]:
from indicnlp.tokenize import indic_tokenize

In [ ]:
# funtion to tokenize hindi text
def hindi_tokenizer(text_in_hindi):
  hindi_tokens=[]
  for token in indic_tokenize.trivial_tokenize(text_in_hindi): 
    hindi_tokens.append(token)
  return hindi_tokens

In [ ]:
#function to tokenize english text
def english_tokenizer(text_in_english):
  english_tokens=[]
  for token in eng.tokenizer(text_in_english): 
    english_tokens.append(token.text)
  return english_tokens

In [ ]:
hindi_field =Field(tokenize = hindi_tokenizer, init_token = '<sos>', eos_token = '<eos>') #field to tokeinze and load 
english_field =Field(tokenize = english_tokenizer, init_token = '<sos>', eos_token = '<eos>')

In [ ]:
fields = [('source', hindi_field), ('target', english_field)]

In [ ]:
#loading the train and validation data
train_ds, val_ds = data.TabularDataset.splits(path = '/content/gdrive/My Drive/Hindi to English MT/',train = 'train_ds.csv', test = 'validation_ds.csv',format = 'csv',
                            fields =[('source', hindi_field), ('target', english_field)])

In [ ]:
len(train_ds)

81857

In [ ]:
hindi_field.build_vocab(train_ds,max_size=20000) #hindi vocabulary
english_field.build_vocab(train_ds,max_size=20000) #english vocabulary

Defining the Encoder architecture

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, layers, dropout_val):
    #input size is equal to hindi vocabulary size and embedding_size is equal to dimensions of embeddings
    super(Encoder, self).__init__()
    self.dropout = nn.Dropout(dropout_val)
    self.embedding = nn.Embedding(input_size,embedding_size)
    #input to LSTM has dimensions equal to embedding_size and output has dimensions equal to hidden_size
    self.lstm = nn.LSTM(embedding_size, hidden_size, layers, dropout = dropout_val)
  
  def forward(self,token_vec):
    #token_vec is a vector of indices mapping a word to its index in the vocabulary
    embedding = self.dropout(self.embedding(token_vec)) #embedding is a 3D tensor of shape (seq length, batch_size, embedding_size)
    outputs, (hidden,cell) = self.lstm(embedding) #the embedding is passed as input to the LSTM
    return hidden, cell #encoder output is not stored; only hidden and cell states are of concern in encoder

Definning the Decoder architecture

In [ ]:
class Decoder(nn.Module):
   def __init__(self, input_size, embedding_size, hidden_size, output_size, layers, dropout_val):
     # here, input_size= size of english vocabulary, embedding_size= dimensions of embedding as defined, hidden_size as defined and output_size=english vocabulary size
     super(Decoder, self).__init__()
     self.dropout = nn.Dropout(dropout_val)
     self.embedding = nn.Embedding(input_size,embedding_size)
     self.lstm = nn.LSTM(embedding_size, hidden_size, layers, dropout = dropout_val) 
     self.fc = nn.Linear(hidden_size, output_size) 

   def forward(self,token_vec,hidden,cell): #token_vec is one dimensional i.e. shape(token_vec) = (batch_size). 
     # However the decoder predicts one word at a time, thus the required dimensions are (1, batch_size). 
     token_vec = token_vec.unsqueeze(0) # unsqueeze adds one more dimension to token_vec
     embedding = self.dropout(self.embedding(token_vec)) #embedding is a 3D tensor of shape (1, batch_size, embedding_size)
     outputs, (hidden,cell) = self.lstm(embedding, (hidden,cell)) # hidden and cell states are used to determine the next word in the sequence and output is the current predicted word. 
     predictions = self.fc(outputs) # shape(outputs)= (1, batch size, hidden size), shape(predictions)= (1, batch_size, english vocabulary size)
     predictions = predictions.squeeze(0) #remove the one extra dimension which was added using unsqueeze. 
     return predictions, hidden, cell

Defining the Seq2Seq class to define the model architecture

In [ ]:
# Now we need to define a class which will define our model.
class seq2seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(seq2seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio = 0.5): #teacher_force_ratio helps in preventing the model from overfitting and underfitting. 
        # teacher_force_ratio helps in deciding whether the next input word to the decoder will be actual/target word or the previous predicted word.
        output_vec = torch.zeros(target.shape[0], source.shape[1], len(english_field.vocab)).to(device)
        hidden, cell = self.encoder(source)
        input_token = target[0]
        for i in range(1, target.shape[0]):
            output, hidden, cell = self.decoder(input_token, hidden, cell) # Size of output = (batch_size, eng vocabulary size)          
            guess_val = output.argmax(1)
            if (random.random() < teacher_force_ratio): #half of the times this will be true if teacher_force_ratio is 0.5
              input_token = target[i]  #in this case next input to the decoder is target/actual word
            else:  input_token= guess_val #in this case next input to the decoder is predicted word
            output_vec[i] = output
        return output_vec

Setting optimal hyperparameters for Training

In [ ]:
#Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs =30
epoch_loss=0.0
layers = 2 # number of neural network layers in rnn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder_input_size = len(hindi_field.vocab) 
decoder_input_size = len(english_field.vocab)
output_size = len(english_field.vocab)
hidden_size = 1024 #encoder and decoder have same hidden size
embedding_size = 250 #encoder and decoder embedding size
dropout = 0.5 #encoder and decoder dropout value

Training the Model

In [ ]:
train_iterator, test_iterator = BucketIterator.splits((train_ds, val_ds),batch_size=batch_size,sort_within_batch=True,sort_key=lambda x: len(x.source),
    device=device,) #creating batches of size 32 for training the model
encoder = Encoder(encoder_input_size, embedding_size, hidden_size, layers, dropout).to(device) #passing the inputs to encoder
decoder = Decoder(decoder_input_size, embedding_size, hidden_size, output_size,layers,dropout,).to(device) #passing the inputs to decoder
model = seq2seq(encoder, decoder).to(device)
pad_index = english_field.vocab.stoi['<pad>'] #finding the index of token <pad> in english vocabulary
criterion = nn.CrossEntropyLoss(ignore_index = pad_index) #padding is being done to make all inputs of same length
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
step = 0
for epoch in range(0,epochs):
    print(f"[Epoch {epoch} / {epochs}]")
    model.eval()
    model.train()
    for id, batch in enumerate(train_iterator):
        input_sentence = batch.source.to(device) # get inputs and get to cuda
        target_sentence = batch.target.to(device) # get target outputs and get to cuda
        output = model(input_sentence, target_sentence) #forward propagation
        output = output[1:].reshape(-1, output.shape[2]) #removing the start token from model's prediction and reshaping it to make it make it fit for input to loss function
        target_sentence = target_sentence[1:].reshape(-1) #removing the start token from actual target translation
        optimizer.zero_grad() 
        loss = criterion(output, target_sentence)
        loss.backward() #backward propagation
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step() #gradient descent. The optimizer iterates over all parameters (tensors) to be updated and their internally stored gradients are used.
        del target_sentence,output,input_sentence
        step += 1
        epoch_loss+=loss.item()
    print("Epoch loss : ", loss.item())
    # print("epoch loss: ", epoch_loss/len(train_iterator))

[Epoch 0 / 30]
Epoch loss :  4.4644012451171875
[Epoch 1 / 30]
Epoch loss :  5.541215419769287
[Epoch 2 / 30]
Epoch loss :  5.179983615875244
[Epoch 3 / 30]
Epoch loss :  5.598097801208496
[Epoch 4 / 30]
Epoch loss :  2.7793753147125244
[Epoch 5 / 30]
Epoch loss :  5.716026306152344
[Epoch 6 / 30]
Epoch loss :  5.463747978210449
[Epoch 7 / 30]
Epoch loss :  2.187171459197998
[Epoch 8 / 30]
Epoch loss :  4.6713643074035645
[Epoch 9 / 30]
Epoch loss :  2.149890422821045
[Epoch 10 / 30]
Epoch loss :  1.905730962753296
[Epoch 11 / 30]
Epoch loss :  3.165379524230957
[Epoch 12 / 30]
Epoch loss :  2.07071852684021
[Epoch 13 / 30]
Epoch loss :  4.8509650230407715
[Epoch 14 / 30]
Epoch loss :  1.6491650342941284
[Epoch 15 / 30]
Epoch loss :  2.4881155490875244
[Epoch 16 / 30]
Epoch loss :  1.7846518754959106
[Epoch 17 / 30]
Epoch loss :  2.103574514389038
[Epoch 18 / 30]
Epoch loss :  1.5608069896697998
[Epoch 19 / 30]
Epoch loss :  2.937772512435913
[Epoch 20 / 30]
Epoch loss :  0.86394828557

In [ ]:
path = "/content/gdrive/My Drive/Hindi to English MT/lstm_phase1.pth"
# torch.save(model,path) #saving the trained model at defined location
# model.train()
model = torch.load(path) #loading the model
model.eval()

seq2seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(20004, 250)
    (rnn): LSTM(250, 1024, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(20004, 250)
    (rnn): LSTM(250, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=20004, bias=True)
  )
)

Function to translate hindi sentences(index vectors) to english sentences(index vector)

In [ ]:
def hin_to_eng_translation(model, hindi_field, english_field, device, hindi_num_vec, max_length=50):
    hindi_tensor = torch.LongTensor(hindi_num_vec).unsqueeze(1).to(device)
    with torch.no_grad():
        hidden, cell = model.encoder(hindi_tensor)
    eng_num_vec = [english_field.vocab.stoi["<sos>"]]
    eos_idx=english_field.vocab.stoi["<eos>"]
    for _ in range(max_length):
        curr_input = torch.LongTensor([eng_num_vec[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(curr_input, hidden, cell)
            curr_output = output.argmax(1).item()
        eng_num_vec.append(curr_output)
        if (curr_output == eos_idx):
            break
    return eng_num_vec

## Testing the Model 
Obtaining the reference and prediction files to compute Bleu score and Meteor Score using Evaluation Script. 

Obtaining the predicted sentences for validation set

In [ ]:
file1 = open("/content/gdrive/My Drive/Hindi to English MT/validation_prediction.txt","w")
csv_file = open("/content/gdrive/My Drive/Hindi to English MT/validation_ds.csv",encoding='utf-8')
rows = csv.reader(csv_file)
for row in rows:
    hindi_sentence = row[0]
    hindi_sentence_token=[]
    if type(hindi_sentence) == str:
      for t in indic_tokenize.trivial_tokenize(hindi_sentence):
        hindi_sentence_token.append(t)
    else:
        for t in hindi_sentence:
          hindi_sentence_token.append(t)
  
    hindi_sentence_token.insert(0, hindi_field.init_token)
    hindi_sentence_token.append(hindi_field.eos_token)
    hindi_num_vec = []
    for t in hindi_sentence_token:
      hindi_num_vec.append(hindi_field.vocab.stoi[t])
    eng_num_vec = hin_to_eng_translation(model, hindi_field, english_field, device, hindi_num_vec, max_length=50)
    english_sentence_list=[]
    for word_idx in eng_num_vec:
      english_sentence_list.append(english_field.vocab.itos[word_idx])
    english_sentence_list.pop(0)
    if (len(english_sentence_list)>2):
      english_sentence=str(english_sentence_list[0][0].upper()) + str(english_sentence_list[0][1:]) +" "
      for string in english_sentence_list[1:-3]:
           english_sentence+=string+' '
      english_sentence+= str(english_sentence_list[-3])+str(english_sentence_list[-2])
    else: 
      english_sentence=str(english_sentence_list[0][0].upper()) + str(english_sentence_list[0][1:])
    file1.write(english_sentence + "\n")
  
file1.close()

Saving the refernce sentences for validation set

In [ ]:
csv_file = open("/content/gdrive/My Drive/Hindi to English MT/validation_ds.csv",encoding='utf-8')
rows = csv.reader(csv_file)
file = open("/content/gdrive/My Drive/Hindi to English MT/validation_english.txt","w")
for row in rows:
    file.write(row[1]+"\n")
file.close()

### Obtaining translation for hindistatements.csv

In [ ]:
file1 = open("/content/gdrive/My Drive/Hindi to English MT/predicted_text.txt","w")
csv_file = open("/content/gdrive/My Drive/Hindi to English MT/hindistatements.csv",encoding='utf-8')
rows = csv.reader(csv_file)
i=0
for row in rows:
    if (i==0): 
      i+=1
      continue
    hindi_sentence = row[2]
    hindi_sentence_token=[]
    if type(hindi_sentence) == str:
      for t in indic_tokenize.trivial_tokenize(hindi_sentence):
        hindi_sentence_token.append(t)
    else:
        for t in hindi_sentence:
          hindi_sentence_token.append(t) 
    hindi_sentence_token.insert(0, hindi_field.init_token)
    hindi_sentence_token.append(hindi_field.eos_token)
    hindi_num_vec = []
    for t in hindi_sentence_token:
      hindi_num_vec.append(hindi_field.vocab.stoi[t])
    eng_num_vec = hin_to_eng_translation(model, hindi_field, english_field, device, hindi_num_vec, max_length=50)
    english_sentence_list=[]
    for word_idx in eng_num_vec:
      english_sentence_list.append(english_field.vocab.itos[word_idx])
    english_sentence_list.pop(0)
    if (len(english_sentence_list)>2):
      english_sentence=str(english_sentence_list[0][0].upper()) + str(english_sentence_list[0][1:]) +" "
      for string in english_sentence_list[1:-3]:
           english_sentence+=string+' '
      english_sentence+= str(english_sentence_list[-3])+str(english_sentence_list[-2])
    else: 
      english_sentence=str(english_sentence_list[0][0].upper()) + str(english_sentence_list[0][1:])
    file1.write(english_sentence + "\n")
file1.close()

## Refrences

1. https://arxiv.org/abs/1409.3215
2. https://medium.com/analytics-vidhya/neural-machine-translation-for-hindi-english-sequence-to-sequence-learning-1298655e334a
